In [1]:
import os
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredURLLoader
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
load_dotenv()


C:\Users\mouhand\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:
doc_ld = DirectoryLoader('OnRoad datasets',glob='./*.pdf',loader_cls=PyPDFLoader)
documents = doc_ld.load()

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mouhand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mouhand\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mouhand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\mouhand\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

text = ""
for d in documents:
    text += d.page_content
text_chunks = text_splitter.split_text(text)


In [25]:
def get_vector_store(text_chunks):
    embeddings =  GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    return vector_store

In [26]:
vector_store = get_vector_store(text_chunks)

In [27]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vector_store.as_retriever(), memory=memory)


C:\Users\mouhand\AppData\Local\Temp\ipykernel_23264\411395932.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = "chat_history", return_messages=True)


In [29]:
conversation_chain({'question':'How to improve DPPH'})

{'question': 'How to improve DPPH',
 'chat_history': [HumanMessage(content='How to improve DPPH', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The provided text outlines a process for identifying the causes of low Deliveries Per Paid Hour (DPPH).  It suggests using the PerfectMile or Caravan dashboards to analyze DPPH performance and identify primary leakages, such as Dispatch Failure Leakage.  By addressing these leakages, DPPH can be improved.  The document also emphasizes the importance of collaboration between Operations and Delivery Service Providers (DSPs) to find solutions and improve Capacity Reliability.  However, it doesn't offer specific recommendations for *how* to improve DPPH beyond identifying the leakages.", additional_kwargs={}, response_metadata={})],
 'answer': "The provided text outlines a process for identifying the causes of low Deliveries Per Paid Hour (DPPH).  It suggests using the PerfectMile or Caravan dashboards to analyze DPPH performanc